In [1]:
%cd '/UltimeTradingBot/Crypto_backtest_tools'
WINDOW_SIZE=10
BUY_PERCENT=1.2
SELL_PERCENT=0.2
MAX_FORCAST_SIZE=5
VERSION=1
TESTING_MOD=False
UPGRAD_MOD=False
JUST_IMPORT_DATA=False
#Normalization_File='w15_NoVol_Normalization.json'
#Model_FileName='w15_NoVol_XcryptoAi_model.hdf5'
ALLHIST_FILE='Results_history.json'
DATA_DIR='/UltimeTradingBot/Data/'
FIRST_NORM_FLAG=True
DATA_DIR='/UltimeTradingBot/Data'
Normalization_File=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Norm_v{VERSION}.json'
Model_FileName=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model_v{VERSION}.hdf5'
DATA_FILE=f'{DATA_DIR}/CSV/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Data_v{VERSION}.csv'
REMOTE_DATA_FILE=f'/gdrive/+DATA+/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Data_v{VERSION}.csv.zip'
METAINFO=f"tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min"
window=WINDOW_SIZE
NORM_FILE=Normalization_File
MODEL_FILE=Model_FileName
Px=40
BUFFER_SIZE=250000*Px
SAMPLE_SIZE=5000*Px
ModelTest="/UltimeTradingBot/Data/tp200_w10_max20min_Model_v2.h5"

#DATA_FILE=DATA_DIR+'w'+str(WINDOW_SIZE)+'_EXTData.csv'



/UltimeTradingBot/Crypto_backtest_tools


In [2]:

import sys
sys.path.append('/UltimeTradingBot/Crypto_backtest_tools')
%cd /UltimeTradingBot/Crypto_backtest_tools
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import time
import matplotlib.pyplot as plt
#import ta
import numpy as np
import gc
import random
from sklearn.utils import shuffle
import seaborn as sns
import tensorflow as tf

from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
#from utilities.custom_indicators import SuperTrend
pd.options.mode.chained_assignment = None  # default='warn'
import gc
gc.collect()    
import ccxt
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 


import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

PDEBUG=True
def pdebug(err):
    if PDEBUG:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

/UltimeTradingBot/Crypto_backtest_tools


/usr/local/lib/python3.9/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/local/lib/python3.9/dist-packages/pandas/core/arrays/masked.py:59: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (
2022-11-12 11:10:07.551316: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-12 11:10:07.551361: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-12 11:10:07.592260: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for pl

### Usefull Global and Config Vars

### importing Data

In [3]:
Binance_USDT_HALAL = [
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "SOL/USDT",
    "LTC/USDT",
    "APE/USDT",
    "XRP/USDT",
    "ICP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ROSE/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "WAVES/USDT",
    "DOGE/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "BTC/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "ZIL/USDT",
    "UST/USDT",
    "ENS/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL
#tf = '1m'
oldest_pair = "BTC/USDT"
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")


In [4]:
#df_list1m["BTC/USDT"].iloc[0:10]

### Validated Fonctions

In [5]:
'''
Binance First Candle Finders
Creslin

Get list of all IDs on binance
Returns the first candle / launch timestamp to the minute for each
'''
import urllib.request
import json
import ccxt

def all_ids():
    # load all markets from binance into a list
    id = 'binance'
    exchange_found = id in ccxt.exchanges
    if exchange_found:
        exchange = getattr(ccxt, id)({})
        markets = exchange.load_markets()
        tuples = list(ccxt.Exchange.keysort(markets).items())

        ids = []
        for (k, v) in tuples:
            ids.append(v['id'])

        return ids

def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]


# Get list of all IDs on binance
def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)

        print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")

In [6]:
try:
    MetaData = pd.read_csv("../Data/MetaData.csv",index_col=0)
except:
    MetaData=get_crypto_metadata(Binance_USDT_HALAL)
    MetaData.to_csv("../Data/MetaData.csv")
#allok = pd.read_csv('D:/+DATA+/allok_w15.csv')

In [7]:
#MetaData.to_csv("D:\+DATA+\MetaData.csv")
pair_list=Binance_USDT_HALAL
window=WINDOW_SIZE
buy_weight=50
sample_size=10000
min_days=MAX_FORCAST_SIZE
buffer_size=100000
#MetaData

In [8]:
def expand_row(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        df["high"+str(i)] = df["high"][i:]
        df["low"+str(i)] = df["low"][i:]
        df["open"+str(i)] = df["open"][i:]
        df["close"+str(i)] = df["close"][i:]
        df["volume"+str(i)] = df["volume"][i:]
    return df

def justlast_remover(df):
    justlast=["BTC_open","BTC_low","BTC_close","open","low","close"]
    for key in df.keys():
        #key.find("-1") != -1 and key.find("open-1") == -1) or
        if ( key.find("close") != -1 ):
            justlast.append(key)
    df=df.drop(columns=justlast)
    return df

def expand_previous(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            #df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i].to_list()

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i].to_list()

            # df.loc[window:len(df),"open-"+str(i)]=None
            # df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i].to_list()           
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i].to_list()            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i].to_list()
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            prerr("Error in     expand_previous: " +str(i))
    if window >= len(df): return df       
    return df.iloc[window:]

def expand_previous_org(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i].to_list()

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i].to_list()

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i].to_list()           
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i].to_list()            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i].to_list()
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            prerr("Error in     expand_previous: " +str(i))
    if window >= len(df): return df       
    return df.iloc[window:]


def expand_previous_err(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i]

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i]
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
    if window >= len(df):
        return df
    return df.iloc[window:]

def expand_timeframe(df_minutes,df_hours, window=2):
    dfm = df_minutes.copy()
    for j in range(1, window+1):
        for i in df_hours[dfm.iloc[0].name:].index:
        #prerr(str(i))
            try:
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"high_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['high']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"low_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['low']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"open_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['open']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"close_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['close']
            except:
                prerr("Error Merging: "+str(i))
    
    return dfm


def float_or_not(var):
    try:
        x=float(var)
    except:
        x=None
    return x

def expand_to_1h(df_1m,df_1h, window=2):
    dfm = df_1m.copy()
    index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
                dfm.loc[timefragment,"high_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'])
                dfm.loc[timefragment,"low_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'])
                dfm.loc[timefragment,"open_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'])
                dfm.loc[timefragment,"close_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_4h(df_1m,df_4h, window=2):
    dfm = df_1m.copy()
    #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
    index_start=df_4h.index.intersection(dfm.index.round(freq='4H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=4*60, freq="min"))
                dfm.loc[timefragment,"high_4h-"+str(j)]=float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['high'])
                dfm.loc[timefragment,"low_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['low'])
                dfm.loc[timefragment,"open_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['open'])
                dfm.loc[timefragment,"close_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_1d(df_1m,df_1d, window=2,time_suffix="1d"):
    dfm = df_1m.copy()
    index_start=df_1d.index.intersection(dfm.index.round(freq='1d'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                prerr(i)
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=24*60, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_5m(df_1m,df_5m, window=2,time_suffix="5m"):
    dfm = df_1m.copy()
    index_start=df_5m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=5, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm


def expand_to_15m(df_1m,df_15m, window=2,time_suffix="15m"):
    dfm = df_1m.copy()
    index_start=df_15m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):    
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=15, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def rapid1d_expand(df1m,df1d,window=2):
    d1min=df1m.copy()
    d1day=df1d.loc[
    d1min.index[0].round(freq='1d')-pd.Timedelta(str(window)+' day'):
    d1min.index[len(d1min)-1].round(freq='1d')+pd.Timedelta('1 day')
    ].copy()
    d1day_pre=expand_previous(d1day,window)
    d1day_pre=d1day_pre.drop(columns=['open', 'low','close','high','volume'])
    d1day_pre=d1day_pre.add_suffix("_day")
    d1min=pd.merge_asof(
        d1min, d1day_pre, on=None, left_on=None, right_on=None, left_index=True, 
        right_index=True, by=None, left_by=None, right_by=None, 
        suffixes=('', '_day'),
        tolerance=pd.Timedelta('1 day'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid1h_expand(df1m,df1h,window=2):
    d1min=df1m.copy()
    d1hour=df1h.loc[
    d1min.index[0].round(freq='H')-pd.Timedelta(str(window)+' hour'):
    d1min.index[len(d1min)-1].round(freq='H')+pd.Timedelta('1 hour')
    ].copy()
    d1hour_pre=expand_previous(d1hour,window)
    d1hour_pre=d1hour_pre.drop(columns=['open', 'low','close','high','volume'])
    d1hour_pre=d1hour_pre.add_suffix("_hour")
    d1min=pd.merge_asof(
    d1min, d1hour_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_hour'),
    tolerance=pd.Timedelta('1 hour'), allow_exact_matches=True, direction='backward')
    return d1min


def rapid5m_expand(df1m,df5m,window=2):
    d1min=df1m.copy()
    d5min=df5m.loc[
    d1min.index[0].round(freq='5 min')-pd.Timedelta(str(window*5+10)+' min'):
    d1min.index[len(d1min)-1].round(freq='5 min')+pd.Timedelta('5 min')
    ].copy()
    d5min_pre=expand_previous(d5min,window)
    d5min_pre=d5min_pre.drop(columns=['open', 'low','close','high','volume'])
    d5min_pre=d5min_pre.add_suffix("_5min")
    d1min=pd.merge_asof(
    d1min, d5min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_5min'),
    tolerance=pd.Timedelta('5 min'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid15m_expand(df1m,df15m,window=2):
    d1min=df1m.copy()
    d15min=df15m.loc[
    d1min.index[0].round(freq='15 min')-pd.Timedelta(str(window*15+30)+' min'):
    d1min.index[len(d1min)-1].round(freq='15 min')+pd.Timedelta('15 min')
    ].copy()
    d15min_pre=expand_previous(d15min,window)
    d15min_pre=d15min_pre.drop(columns=['open', 'low','close','high','volume'])
    d15min_pre=d15min_pre.add_suffix("_15min")
    d1min=pd.merge_asof(
    d1min, d15min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_15min'),
    tolerance=pd.Timedelta('15 min'), allow_exact_matches=True, direction='backward')
    return d1min


def full_expand(df1m,df5m,df15m,df1h,df1d,window=10):
    d1min=df1m.copy()
    d1min=expand_previous(d1min,window=window).drop(columns=["volume"])
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    return d1min

def full_expand_org(df1m,df5m,df15m,df1h,df1d,window=10):
    d1min=df1m.copy()
    d1min=expand_previous(d1min,window=window)
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    return d1min


def day_expand(data_full):
    ser = pd.to_datetime(pd.Series(data_full.index))
    data_full["day"]=ser.dt.isocalendar().day.values
    data_full["hour"]=ser.dt.hour.values
    data_full["minute"]=ser.dt.minute.values

# merging
def pair_btc(pair="LTC/USDT",window=2):
    Pair_Full=full_expand(df_list1m[pair],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[df_list1m[pair].iloc[0].name:
        df_list1m[pair].iloc[len(df_list1m[pair])-1].name],
        df_list5m["BTC/USDT"],df_list15m["BTC/USDT"],df_list1h["BTC/USDT"],df_list1d["BTC/USDT"],window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='outer',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged






In [9]:
def buy_results(df,min_pourcent=BUY_PERCENT):
    mino=min_pourcent*0.01
    df["buy"]=(
        ((df["high"].shift(periods=1, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| ((
          df["high"].shift(periods=2, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| ((
          df["high"].shift(periods=3, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)
    ).replace({False: 0, True: 1}) 
 
def buy_results_gen(df,min_pourcent=BUY_PERCENT,window=3):
    mino=min_pourcent*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    print(code)
    exec(code)

def buy_sell(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    mino=buy_pourcent*0.01
    maxo=-sell_pourcent*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    codep1='df["sell"]=((df["buy"]==0)&(( '
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )& (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    df["bs"]=((df['buy']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})

def buy_only(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    mino=buy_pourcent*0.01
    maxo=-sell_pourcent*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    # codep1='df["sell"]=((df["buy"]==0)&(( '
    # for i in range(1,window):
    #     codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )& (('
    # codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
    # code=codep1+codep2
    # prerr(code)
    # exec(code)
    # df["bs"]=((df['buy']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
    return df

In [10]:
def Meta_expand(data_full,metadt,pair):
    data_full["lunch_day"]=int(-(pd.to_datetime(metadt[metadt["Pair"] == pair]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)

def mini_expand(pair="LTC/USDT",i=0,j=10000,window=2):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged

def mini_expand2(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=7)
    return Merged

def mini_expand3(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=3):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='1 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='5 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list15m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='15 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1h["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 H'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1d["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 d'):Pair_Full.iloc[len(Pair_Full)-1].name],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    # Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
    #         right_index=True, suffixes=('', ''))
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='inner',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    #buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    buy_only(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    # Merged=justlast_remover(Merged)
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    return Merged

def mini_expand3old(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=3):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    # Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    Merged=justlast_remover(Merged)

    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    return Merged



def slow_expand(pair="LTC/USDT",i=0,j=100000,window=3):
    df=mini_expand(pair=pair,i=i,j=j,window=window)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window)],axis=0)
    return df

def pair_data_gen(pair="LTC/USDT",i=0,j=100000,window=3,metadata=MetaData):
    df=mini_expand3(pair=pair,i=i,j=j,window=window,metadata=metadata)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand3(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window,metadata=metadata)],axis=0)
        # Meta_expand(df,metadata,pair)
        # buy_sell(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=7)
        #print("loop "+str(mx)+"--> size of df: "+str(len(df)))
    return df

###

def data_is_enough(df,days=10,window=10):
    if days <= window:
        return df[~df.isnull().any(axis=1) |(df["open-"+str(days)+"_day"].isnull() & ~df["open-"+str(window-1)+"_hour"].isnull()  & ~df["open-"+str(window-1)+"_5min"].isnull() & ~df["open-"+str(days-1)+"_day"].isnull())]
    else:
        prerr("number of days must be equal or lower than window")
        return df

def data_cleanup(df):
    return df.dropna()
    #return df[~df.isnull().any(axis=1)]
    
def data_shufler2(df):
    return df.sample(frac=1).reset_index()
    
def data_shufler(df):
    x = len(df)
    df["num_index"] = range(0, x, 1)
    df.set_index(df['num_index'], inplace=True)
    df = df.reindex(np.random.permutation(df.index))
    try:df= df.drop("num_index",axis=1)
    except:pass
    #df = df.reindex(np.random.permutation(df.index))
    return df
    
def data_np_shufler(df):
    df = shuffle(df)
    #df = df.reindex(np.random.permutation(df.index))
        



def data_chooser(df,weight=50,row_numbers=100000):
    df=data_shufler(df)
    if row_numbers>=len(df):
        row_numbers=len(df)
    df=pd.concat([df[df["buy"]==1].iloc[:int(row_numbers*weight*0.01)],
                 df[df["buy"]==0].iloc[:int(row_numbers*(100-weight)*0.01)]])
    df=data_shufler(df)
    return df
    

def data_looper(pair_list=Binance_USDT_HALAL,window=15,buy_weight=50,sample_size=100000,min_days=10,buffer_size=100000):
    xdf=pd.DataFrame()
    for pair in pair_list:
        if pair != "BTC/USDT":
            print("working on: "+pair)
            df=pair_data_gen(pair=pair,i=0,j=buffer_size,window=window)
            gc.collect()

            df=data_is_enough(df,days=min_days,window=window)
            gc.collect()

            df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
            gc.collect()

            print(pair+" is processed")
            xdf=pd.concat([xdf,df],axis=0)
            del(df)
            gc.collect()
        else:
            print("ignore BTC")
    return xdf

def data_looper_fast(pair_list=Binance_USDT_HALAL,window=15,buy_weight=50,sample_size=100000,min_days=10,buffer_size=100000):
    xdf=pd.DataFrame()
    count=0
    for pair in pair_list:
        if pair != "BTC/USDT":
            print("working on: "+pair ,end=" -->")
            try:
                df=pair_data_gen(pair=pair,i=0,j=buffer_size,window=window)
                gc.collect()
                count+=1
                # df=data_is_enough(df,days=min_days,window=window)
                # gc.collect()
                df.reindex(np.random.permutation(df.index))
                df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
                gc.collect()

                df=data_cleanup(df)
                print(pair+" is processed")
            except:
                print(f"error while processing {pair} {count}/{len(pair_list)}")
            xdf=pd.concat([xdf,df],axis=0)
            del(df)
            gc.collect()
        else:
            print("ignore BTC")
            
    return xdf

def volume_cleaner(df):
    VolRemover=["volume","volume-1","BTC_volume","BTC_volume-1"]
    for key in df.keys():
        if key.find("volume-1_") != -1 :
            VolRemover.append(key)
    df=df.drop(columns=VolRemover)
    return df
    
Normalization=None
def normalize(dataset,file=Normalization_File):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        #print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        #print('normalization is loaded')
        pass

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

## buy or sell mod

In [11]:
# Costumaize buy condition here
def buy_only(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    try:
        
        ## test param
        #buy_pourcent=1
        window=15
        print (f"---buy_only--- Buy percent: {buy_pourcent}%")
        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b"]=((('
        for i in range(1,window):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)


##################### debug ###############33
        strcomment='''
        codep1='df["b5"]=((('
        for i in range(1,5):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)



        codep1='df["b10"]=((('
        for i in range(1,10):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        codep1='df["b15"]=((('
        for i in range(1,15):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)


        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b30"]=((('
        for i in range(1,30):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)


        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b60"]=((('
        for i in range(1,60):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b180"]=((('
        for i in range(1,180):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)
    
        '''#####

        # codep1='df["b5p"]=((('
        # for i in range(1,window+5):
        #     codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=0.005 )| (('
        # codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=0.005)).replace({False: 0, True: 1})'
        # code=codep1+codep2
        # prerr(code)
        # exec(code)

        # codep1='df["b9p"]=((('
        # for i in range(1,window+9):
        #     codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=0.009 )| (('
        # codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=0.009)).replace({False: 0, True: 1})'
        # code=codep1+codep2
        # prerr(code)
        # exec(code)

        # codep1='df["b15p"]=((('
        # for i in range(1,window+15):
        #     codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=0.015 )| (('
        # codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=0.015)).replace({False: 0, True: 1})'
        # code=codep1+codep2
        # prerr(code)
        # exec(code)






        # codep1='df["sell"]=((( '
        # for i in range(1,window):
        #     codep1=codep1+'df["low"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )& (('
        # codep2='df["low"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
        # code=codep1+codep2
        # prerr(code)
        # exec(code)

        # df["buy"]=((df['b']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
        
        




        #df.pop("b")
        #df.pop("sell")
        
        
        # df["f1"]=(df["high"].shift(periods=-1, freq=None, axis=0, fill_value=None)-df["high"])/df["high"]
        # df["f2"]=(df["high"].shift(periods=-2, freq=None, axis=0, fill_value=None)-df["high"])/df["high"]
        # df["f15_p"]=df["high-1_15min"].shift(periods=-2, freq=None, axis=0, fill_value=None)
        # df["fh_p"]=df["high-1_hour"].shift(periods=-2, freq=None, axis=0, fill_value=None)

        # df['signal'] = np.where(
        # df['high'] >= df.shift(1).rolling(window)['high'].min(), 1,
        # np.where(
        #     df['low'] >= df.shift(1).rolling(window)['low'].min(), -1,
        #     0
        # )
        # )




    #     df['ismin'] = np.where(
    #     df['high'] <= df.shift(-window-1).rolling(2*window)['high'].min(), 1,
    #    0
    #     )
    #     df["buy"]=((df['buy']==1 ) & (df['ismin']==1)).replace({False: 0, True: 1})
    #     df.pop("b")
    #     df.pop("ismin")

        

    #     df['ismax'] = np.where(
    #     df['low'] >= df.shift(1).rolling(window)['low'].min(), 1,
    #    0
    #     )
        
        
        codep1='df["sell"]=((( '
        for i in range(1,window):
            codep1=codep1+'df["low"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo ) | (('
        codep2='df["low"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        df["buy"]=((df['b']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
        
        

    except Exception as e:
        print("Error buy only")
        print(e)
    p=df.pop("b")
    p=df.pop("sell")

    return df

# Test space

In [ ]:
WINDOW_SIZE=10
BUY_PERCENT=1.2
SELL_PERCENT=0.2
MAX_FORCAST_SIZE=5


In [ ]:
dddd=pd.DataFrame()
for pair in ["GMT/USDT","XRP/USDT","RVN/USDT","ETC/USDT","LUNA/USDT","APE/USDT","ETH/USDT","DOGE/USDT"]:
    print( f'===  {pair}  === ')
    dtest=mini_expand3(pair=pair,i=0,j=len(df_list1m[pair]),window=1,metadata=MetaData)
    #print(f"  - {BUY_PERCENT}% / {MAX_FORCAST_SIZE}min ==b 0.27%=> " +str({dtest["buy"].mean()*100}))

    print(f"  - {MAX_FORCAST_SIZE}min ==buy {BUY_PERCENT}%=> " +str( format(float(dtest["buy"].mean()*100) ,".2f")))
    #print(f"  - {MAX_FORCAST_SIZE}min ==sell {SELL_PERCENT}%=> " +str( format(float(dtest["sell"].mean()*100) ,".2f")))


    dddd=pd.concat([dtest,dddd],axis=0)


print( f'=== ALL === ')
  
#print(f"  - {MAX_FORCAST_SIZE}min ==b {SELL_PERCENT}%=> " +str(format(float(dddd["sell"].mean()*100), ".2f")))
print(f"  - final buy dessision {MAX_FORCAST_SIZE}min  -  {BUY_PERCENT}% => " + str( format(float(dddd["buy"].mean()*100), ".2f")))


  
  
  
  
  
    # print(f"  - 0.9% / 11min => " +str( {dtest["b9p"].mean()*100}))
    # print(f"  - 0.5% / 7min => " +str( {dtest["b5p"].mean()*100}))

In [ ]:
dddd.buy.mean()

In [ ]:
pair='ETH/USDT'
pt=mini_expand3(pair,i=0,j=int(len(df_list1m[pair])/2),window=10,metadata=MetaData)

In [ ]:
np.delete(dt, dt[:,1], axis=None)

In [ ]:
buyy=pt.pop("buy")
bb=pt.pop("b")
bprice=pt.pop("price")

In [ ]:
dt=pt.to_numpy()

In [ ]:
Y=(dt[:,-1]-1)*(-1)
dt[:,-1]=Y

In [ ]:
from keras.models import load_model 
model2=load_model("../dontbuy_model_w10_noprice_p1.2.h5")

In [ ]:
predo=(model2.predict( tf.convert_to_tensor(np.nan_to_num(dt[:, 0:-1],copy=False,nan=0) ,dtype=tf.float32))).round().transpose()[0]
False_prediction=dt[:,-1][(predo!=Y[:])]
True_prediction=dt[:,-1][(predo==Y[:])]
fp0=False_prediction[False_prediction==0].shape[0]*100/dt.shape[0] #### it mean that 11.73 of the 0 class are wrong the buyed witch lead the losses
print( f"Prediction mean :{predo.mean()*100} %"+'#### the colose to 50 the better')
print( f"False prediction class 0 :{fp0} %"+'#### it mean that x of the 0 class are wrong the buyed witch lead the losses')
fp1=False_prediction[False_prediction==1].shape[0]*100/dt.shape[0]   #### we don't buy x % of the correct chanses
print( f"False prediction class 1 :{fp1} %"+'#### we don\'t buy x % of the correct chanses')
trp0=True_prediction[True_prediction==0].shape[0]*100/dt.shape[0]   #### it mean that 40 % of class 0 are predicted correctly
trp1=True_prediction[True_prediction==1].shape[0]*100/dt.shape[0]     #### the only buying correct of all
print( f"True prediction class 0 :{trp0} %"+'#### it mean that 40 % of class 0 are predicted correctly')
print( f"True prediction class 1 :{trp1} %"+'#### the only buying correct of all')
print(f"successful buy pourcent of unsuccessfull: {100*trp1/(trp1+fp1)}  %")


In [ ]:
pt.sell=(pt.sell-1)*(-1)

In [ ]:
price=pt.pop("price")

In [ ]:
dt=pt.to_numpy()

In [ ]:
from keras.models import load_model
model2=load_model("../dontbuy_model_w10_noprice_p1.2.h5")

In [ ]:
mm.predict()

In [ ]:
gc.collect()

## garbage test

In [ ]:

etc=mini_expand3(pair="BTC/USDT",i=0,j=500000,window=8,metadata=MetaData)

In [ ]:

etc.buy.mean()

In [ ]:
etc_np=etc.iloc[:,0:-10].reset_index().reset_index().drop(columns="date").to_numpy()

In [ ]:
np.nan_to_num(etc_np)

etc_np=tf.convert_to_tensor(etc_np, dtype=tf.float32)

In [ ]:
model=tf.keras.models.load_model(Model_FileName)
etc["prediction"]=model.predict(etc_np).round()

In [ ]:
etc[(etc.b60==0)&(etc.prediction==1)]

In [ ]:
round(etc.iloc[:2,0:-11])

In [ ]:
etc[0:1]

In [ ]:
del etc

# Start Data genration Process

In [ ]:
if  False:
    ww=WINDOW_SIZE
    gc.collect()
    df=data_looper_fast(pair_list=Binance_USDT_HALAL,window=WINDOW_SIZE,buy_weight=50,sample_size=SAMPLE_SIZE,min_days=0,buffer_size=BUFFER_SIZE)
    # Test Data:
    # df=data_looper_fast(pair_list=Binance_USDT_HALAL[50:80],window=WINDOW_SIZE,buy_weight=10,sample_size=SAMPLE_SIZE*3,min_days=0,buffer_size=BUFFER_SIZE)
    gc.collect()
    #df=volume_cleaner(df)
    #df=data_shufler(df)
    #df=df.reindex(np.random.permutation(df.index))
    #df=df.reindex(np.random.permutation(df.index))
    time.sleep(3)
    gc.collect()
    print('################### reindexing and shufling ###############')
    df=df.reset_index().drop(columns="num_index")
    gc.collect()
    for i in range(1):
        df = df.reindex(np.random.permutation(df.index)).reset_index().drop(columns="index")
        gc.collect()
    print('################### Cleaning Null values ###############')
    df=df.dropna()
    print(df)
# else:
#     if TESTING_MOD:
#         df_testing=data_looper_fast(pair_list=Binance_USDT_HALAL[70:90],window=WINDOW_SIZE,buy_weight=10,sample_size=SAMPLE_SIZE,min_days=0,buffer_size=BUFFER_SIZE)
#         df_testing=df_testing[~df_testing.isna().any(axis=1)]

In [ ]:
#pair_list=Binance_USDT_HALAL[:20]
pair_list=Binance_USDT_HALAL[0:55]
window=WINDOW_SIZE
buy_weight=50
sample_size=50000
min_days=1
buffer_size=BUFFER_SIZE
xdf=pd.DataFrame()
count=0
for pair in pair_list:
    if pair != "BTC/USDT":
        print("working on: "+pair ,end=" -->")
        try:
            df=mini_expand3(pair=pair,i=0,j=len(df_list1m[pair]),window=window,metadata=MetaData)
            gc.collect()
            count+=1
            df=df.reset_index()
            try:df.pop("num_index")
            except: pass
            try:df.pop("index")
            except: pass
            try:df.pop("date")
            except: pass
            df=data_shufler(df)            
            df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
            gc.collect()
            df=data_cleanup(df)
            print(pair+f" is processed -- {count}/{len(pair_list)}")
        except Exception as e:
            print(f"error while processing {pair} {count}/{len(pair_list)}")
            print(e)
        xdf=pd.concat([xdf,df],axis=0)
        del(df)
        gc.collect()
df=xdf
del xdf
for i in range(1):
    try:df=df.reset_index() 
    except: pass
    try:df.pop("num_index")
    except: pass
    try:df.pop("index")
    except: pass
    df=df.reset_index()    
    gc.collect()
    df = df.reindex(np.random.permutation(df.index))
    try:df.pop("index")
    except: pass
    gc.collect()
    df=df.reset_index()
try:df.pop("index")
except: pass
    
gc.collect()


In [ ]:
df.to_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_p1.fea")

In [ ]:
try:del df
except:pass
gc.collect()
pair_list=Binance_USDT_HALAL[55:]
window=WINDOW_SIZE
buy_weight=50
sample_size=50000
min_days=1
buffer_size=BUFFER_SIZE
xdf=pd.DataFrame()
count=0
for pair in pair_list:
    if pair != "BTC/USDT":
        print("working on: "+pair ,end=" -->")
        try:
            df=mini_expand3(pair=pair,i=0,j=len(df_list1m[pair]),window=window,metadata=MetaData)
            gc.collect()
            count+=1
            df=df.reset_index()
            try:df.pop("num_index")
            except: pass
            try:df.pop("index")
            except: pass
            try:df.pop("date")
            except: pass
            df=data_shufler(df)            
            df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
            gc.collect()
            df=data_cleanup(df)
            print(pair+f" is processed -- {count}/{len(pair_list)}")
        except Exception as e:
            print(f"error while processing {pair} {count}/{len(pair_list)}")
            print(e)
        xdf=pd.concat([xdf,df],axis=0)
        del(df)
        gc.collect()
df=xdf
del xdf
for i in range(1):
    try:df=df.reset_index() 
    except: pass
    try:df.pop("num_index")
    except: pass
    try:df.pop("index")
    except: pass
    df=df.reset_index()    
    gc.collect()
    df = df.reindex(np.random.permutation(df.index))
    try:df.pop("index")
    except: pass
    gc.collect()
    df=df.reset_index()
try:df.pop("index")
except: pass
    
gc.collect()


In [ ]:
df.to_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_p2.fea")

In [ ]:
df1=pd.read_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_p1.fea")

In [ ]:
df2=pd.read_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_p2.fea")

In [ ]:
df.to_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_full.fea")

In [ ]:
df=pd.read_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_full.fea")

In [4]:
def get_from_fea(file=f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_full.fea",nrows=2000000,srows=0):
    return pd.read_feather(file).drop(columns=["index","price"])[srows:nrows+srows].copy()
dt=get_from_fea(file=f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_full.fea",nrows=500000,srows=0).to_numpy(dtype=np.float32)

In [ ]:
np.save(f"../Data/np/w{WINDOW_SIZE}_buy{BUY_PERCENT}_full_normalized.npy",dt)

In [ ]:
dt=np.load(f"../Data/np/w{WINDOW_SIZE}_buy{BUY_PERCENT}_full_normalized.npy",allow_pickle=True)

In [ ]:
np.nan_to_num(dt, copy=False, nan=0.0, posinf=None, neginf=None)

In [ ]:
gc.collect()

In [ ]:
df.pop("index")
prices=df.pop("price")

In [ ]:
gc.collect()

In [ ]:
if  not JUST_IMPORT_DATA:
    print("saveing the file")
    df.to_csv(DATA_FILE+"buy07.csv",index=False)
    # dffull=df
    print(df)

In [ ]:
df[111:121]

In [ ]:
df.buy.mean()

In [ ]:
if JUST_IMPORT_DATA:
    df=pd.read_csv(DATA_FILE,nrows=20000)
    # df=dffull

In [ ]:
df[df.price==4724.285]

In [ ]:
dtest=mini_expand3(pair="ETH/USDT",i=0,j=len(df_list1m["ETH/USDT"]),window=1,metadata=MetaData)

In [ ]:
dtest[dtest.price==4724.285]

In [ ]:
df.shape

In [ ]:
df=df.dropna()

In [ ]:
gc.collect()

In [ ]:
df.buy.mean()

In [ ]:
dt=pd.read_csv(DATA_FILE).dropna().drop(columns="price").to_numpy() # no price


In [ ]:
orgY=dt[:,-1].copy()
dt[:,-1]=(dt[:,-1]-1)*(-1)

# Part 2: Training


## Testing impoted DATA

# starting numpy process

## convert Pandas DataFrame to numpy object

In [ ]:
#df=df.dropna()
gc.collect()
dt = df.to_numpy()
#dt=tf.convert_to_tensor(dt, dtype=tf.float32)
#dt = df.iloc[0:int(len(df/1.5))].to_numpy()
#dt = np.concatenate((dt,df.iloc[int(len(df/2)):].to_numpy()),axis=0)


In [ ]:
del df

In [ ]:
gc.collect()

In [ ]:
# goodcsv='/UltimeTradingBot/Data/csv/W20F4minP35.csv'

# dt=pd.read_csv(goodcsv).sample(frac=1,random_state=2).to_numpy()

In [ ]:
# del dffull
# df=df.iloc[int(len(df/1.5)):]
# gc.collect()
print(dt.shape)

In [ ]:
dt[:,-1].mean()

In [ ]:
np.nan_to_num(dt)

In [ ]:
dt.shape



In [ ]:
#dt=np.genfromtxt('np_shuffled_cryptodata_w15.csv', delimiter=',')
index_20percent= int(0.5*df.shape[0])
print(index_20percent)
# XVALIDATION= df.iloc[:index_20percent, :-1]
# YVALIDATION= df.iloc[:index_20percent,-1]
# XTRAIN= df.iloc[index_20percent:, 0:-1]
# YTRAIN= df.iloc[index_20percent:,-1]

In [ ]:
#### PANDAS DIRECT NORMALIZATION #####
if True:
    #if FIRST_NORM_FLAG:
    if True:
        print("normalizing ...")
        mean = df.iloc[index_20percent:, 0:-1].mean(axis=0)
        gc.collect()
        std = df.iloc[index_20percent:, 0:-1].std(axis=0)
        gc.collect()


        # df.iloc[:, 0:-1] -= mean 
        # df.iloc[:, 0:-1] /= std

        # XVALIDATION -=mean
        # XVALIDATION /= std
        FIRST_NORM_FLAG=False
        ######################### SAVIN NORM ################
        try:
            Normalization={"mean":mean.tolist(),"std":std.tolist()}
            with open(Normalization_File, 'w+') as fp:
                        json.dump(Normalization, fp,  indent=4)
                        print(Normalization_File)
        except:
            print("error Normalization in juppiter")
    else:print("already normalized")



In [ ]:
with open(Normalization_File) as json_file:
                print(Normalization_File)
                Normalization = json.load(json_file)
                mean=np.array(Normalization["mean"])
                std=np.array(Normalization["std"])

In [ ]:
std.shape

In [ ]:

gc.collect()
df.iloc[:1000000, 0:-1] -= Normalization["mean"]
gc.collect()

df.iloc[:1000000, 0:-1] /= Normalization["std"]

In [ ]:
df.to_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_noprice_normalized.fea")

In [5]:
index_20percent= int(0.5*len(dt[:,0]))
print(index_20percent)

250000


In [6]:
XVALIDATION= dt[:index_20percent, :-1]
YVALIDATION= dt[:index_20percent,-1]


In [7]:
XTRAIN= dt[index_20percent:, 0:-1]
YTRAIN= dt[index_20percent:,-1]

In [ ]:
dt[0,0]

In [ ]:
dt[0,0]

In [ ]:
del dt

In [ ]:
df[0:1]

## Normalization

In [ ]:
import gc
gc.collect()

Tenderalization (mean normalization)

In [ ]:

if True:
    #if FIRST_NORM_FLAG:
    if True:
        print("normalizing ...")
        mean = XTRAIN.mean(axis=0)
        std = XTRAIN.std(axis=0)



        XTRAIN -= mean 
        XTRAIN /= std

        XVALIDATION -=mean
        XVALIDATION /= std
        FIRST_NORM_FLAG=False
        ######################### SAVIN NORM ################
        try:
            Normalization={"mean":mean.tolist(),"std":std.tolist()}
            with open(Normalization_File, 'w+') as fp:
                        json.dump(Normalization, fp,  indent=4)
                        print(Normalization_File)
        except:
            print("error Normalization in juppiter")
    else:print("already normalized")



In [ ]:
dt[:,:-1] -= mean 
dt[:,:-1] /= std

In [ ]:
with open(Normalization_File, 'w+') as fp:
        json.dump(Normalization, fp,  indent=4)
        print(Normalization_File)

In [10]:
################## V2 Testing ######################

TESTING_MOD=True
# Normalization
if TESTING_MOD:
    with open(Normalization_File) as json_file:
                print(Normalization_File)
                Normalization = json.load(json_file)
                mean=np.array(Normalization["mean"])
                std=np.array(Normalization["std"])
# mean=np.array(Normalization["mean"])

if  TESTING_MOD:
    if FIRST_NORM_FLAG:
        print("normalizing ...")
        dt[:,0:-1] -= Normalization["mean"] 
        dt[:,0:-1] /= Normalization["std"] 

    
        FIRST_NORM_FLAG=False
        FIRST_NORM_FLAG=False
    else:print("already normalized")


/UltimeTradingBot/Data/tp120_w10_max5min_Norm_v1.json
normalizing ...


In [ ]:
print(Normalization_File)

In [ ]:
print(XTRAIN.shape)
print(YTRAIN.shape)
print(XVALIDATION.shape)
print(YVALIDATION.shape)

# Neural network model

## Model Test

## Compile the model 

In [ ]:
# if True:
#     IN_DIM=len(mean)
# #     model = Sequential()
# #     model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='relu'))
# #     model.add(Dropout(0.5))
# #     model.add(Dense(int(IN_DIM/4),activation='relu'))
# # #    model.add(Dropout(0.7))
# #     model.add(Dense(8,activation='relu'))
# # #    model.add(Dropout(0.7))
# #     model.add(Dense(1,activation='sigmoid'))
#     model = Sequential()
#     model.add(Dense(IN_DIM,input_dim=IN_DIM,activation='relu'))
#     model.add(Dense(100,activation='tanh'))
#     model.add(Dense(100,activation='relu'))
#     model.add(Dense(40,activation='relu'))
#     model.add(Dense(40,activation='relu'))
#     model.add(Dense(40,activation='relu'))
# #    model.add(Dense(10,activation='relu'))
# #    model.add(Dropout(0.5))
# #    model.add(Dense(20,activation='relu'))
# #    model.add(Dropout(0.7))
#     model.add(Dense(4,activation='relu'))
# #    model.add(Dropout(0.7))
#     model.add(Dense(1,activation='sigmoid'))
#     print(model.summary())
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath=Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='accuracy',mode='auto',patience=50,verbose=1)
#     history = model.fit(XTRAIN,
#                     YTRAIN,
#                     validation_data=(XVALIDATION,YVALIDATION),
#                     epochs=6000,
#                     batch_size=128*50,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#     #94%

In [ ]:
from tensorflow.keras.models import load_model
model2=load_model(Model_FileName+f"1.dropout0.5.onelayer.h5")


In [ ]:
model2.evaluate(XTRAIN,YTRAIN)

In [8]:
Y=dt[:,-1]

In [11]:
del model2

In [ ]:
PRED=model2.predict(dt[:,:-1]).round()

In [ ]:
REAL_PRED=np.array(((PRED[:,0]==1) & (Y==1)),dtype=float)

In [ ]:
REAL_PRED.mean()

In [ ]:
dt[:,-1]=REAL_PRED

In [ ]:
dt[:,-1].mean()

In [ ]:
del model2

In [13]:
for ii in range(2):
#if True:    
    tv=ii
    try : 
      PRED=model2.predict(dt[:,:-1]).round()
      REAL_PRED=np.array(((PRED[:,0]==1) & (Y==1)),dtype=float)
      dt[:,-1]=REAL_PRED
      print(f"--------------------Real prediction mean: {REAL_PRED.mean()} ---------------------------")
      del model2
      del history
    except: print("first time or error")
    gc.collect()
    if True:
        IN_DIM=dt.shape[1]-1
        model2 = Sequential()
        model2.add(Dense(int(100),input_dim=IN_DIM,activation='relu')) #( 100=>66.23)
        # resultus withe 250 d0.3 50 d 20 d 8 = 65.6% vs  65.49 no droppout vs 65.78 d0.5 vs 65.68 d0.4 # 65.48 # one dropout of   0.5 = 66.11   #tanh 66.33 accuracy #softmax 66.12 #softplus 66.5 # sigmoid 66.21 / 
        model2.add(Dropout(0.5))
        model2.add(Dense(int(20),activation='relu')) # best softplus 66.26(69) vs relu 66.65(70)  / -20 => 66.25 /250 ->65.78
        #model2.add(Dropout(0.1)) #5=> 66.00  66.21 vs no dopout 66.28
        #model2.add(Dense(int(20),activation='relu')) # disabled 6.24
        model2.add(Dense(int(5),activation='relu')) # -4 -> -> 66.24
        model2.add(Dense(1,activation='sigmoid'))
        print(model2.summary())
        model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        callbacks_a = ModelCheckpoint(filepath =MODEL_FILE+f"{tv}.multi_train.h5",monitor ='val_accuracy',save_best_only = True, save_weights = True)
        callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=15,verbose=1)
        history = model2.fit(dt[index_20percent:, 0:-1],
                        dt[index_20percent:,-1],
                        validation_data=(dt[:index_20percent, :-1],Y[:index_20percent]),
                        epochs=6000,
                        batch_size=100000,
                        callbacks=[callbacks_a,callbacks_b])

        print('##########################################################################')
        print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
        #94%

first time or error
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               40600     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 20)                2020      
                                                                 
 dense_10 (Dense)            (None, 5)                 105       
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 42,731
Trainable params: 42,731
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/6000
3/3 [=======

In [ ]:
dt.shape[1]

In [14]:
prediction2=(model2.predict( dt[:, 0:-1])).round()

15625/15625 [==============================] - 24s 2ms/step


In [15]:
False_prediction=dt[:index_20percent,-1][(prediction2[:index_20percent].transpose()[0]!=Y[:index_20percent])]
True_prediction=dt[:index_20percent,-1][(prediction2[:index_20percent].transpose()[0]==Y[:index_20percent])]

In [16]:
fp0=False_prediction[False_prediction==0].shape[0]*100/index_20percent #### it mean that 11.73 of the 0 class are wrong the buyed witch lead the losses
print( f"Prediction mean :{prediction2.mean()*100} %"+'#### the colose to 50 the better')
print( f"False prediction class 0 :{fp0} %"+'#### it mean that x of the 0 class are wrong the buyed witch lead the losses')
fp1=False_prediction[False_prediction==1].shape[0]*100/index_20percent   #### we don't buy x % of the correct chanses
print( f"False prediction class 1 :{fp1} %"+'#### we don\'t buy x % of the correct chanses')
trp0=True_prediction[True_prediction==0].shape[0]*100/index_20percent   #### it mean that 40 % of class 0 are predicted correctly
trp1=True_prediction[True_prediction==1].shape[0]*100/index_20percent     #### the only buying correct of all
print( f"True prediction class 0 :{trp0} %"+'#### it mean that 40 % of class 0 are predicted correctly')
print( f"True prediction class 1 :{trp1} %"+'#### the only buying correct of all')
print(f"successful buy pourcent of unsuccessfull: {100*trp1/(trp1+fp1)}  %")


Prediction mean :0.0 %#### the colose to 50 the better
False prediction class 0 :0.0 %#### it mean that x of the 0 class are wrong the buyed witch lead the losses
False prediction class 1 :0.286 %#### we don't buy x % of the correct chanses
True prediction class 0 :99.714 %#### it mean that 40 % of class 0 are predicted correctly
True prediction class 1 :0.0 %#### the only buying correct of all
successful buy pourcent of unsuccessfull: 0.0  %


In [ ]:
model2.evaluate(dt[:index_20percent,:-1], Y[:index_20percent]) #3-> 66.76 / 4 -> 65.75


In [ ]:
model2.save("../buy_model_w10_tp1.2.h5")

In [ ]:
# #minimodel
# if True:
#     IN_DIM=len(mean)
#     model = Sequential()
#     model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='tanh'))
#     model.add(Dense(int(IN_DIM/3),activation='tanh'))
#     model.add(Dense(int(IN_DIM/1),activation='relu'))
#     model.add(Dense(1,activation='sigmoid'))
#     print(model.summary())
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath=Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='accuracy',mode='auto',patience=50,verbose=1)
#     history = model.fit(XTRAIN,
#                     YTRAIN,
#                     validation_data=(XVALIDATION,YVALIDATION),
#                     epochs=6000,
#                     batch_size=128*500,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
    

In [ ]:
# model.save(Model_FileName+"minx.h5")

In [ ]:
# #minimodel
# if True:
#     IN_DIM=len(mean)
#     model3 = Sequential()
#     model3.add(Dense(int(25),input_dim=IN_DIM,activation='tanh'))
#     model3.add(Dense(int(10),activation='relu'))
#     model3.add(Dense(int(10),activation='relu'))
#     model3.add(Dense(1,activation='sigmoid'))
#     print(model3.summary())
#     model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath=Model_FileName+"fast.h5",monitor ='val_accuracy',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='accuracy',mode='auto',patience=50,verbose=1)
#     history = model3.fit(XTRAIN,
#                     YTRAIN,
#                     validation_data=(XVALIDATION,YVALIDATION),
#                     epochs=6000,
#                     batch_size=128*500,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
    

In [ ]:
# #minimodel
# if True:
#     IN_DIM=len(mean)
#     model3 = Sequential()
#     model3.add(Dense(int(IN_DIM),input_dim=IN_DIM,activation='tanh'))
#     model3.add(Dense(int(40),activation='relu'))
#     model3.add(Dense(int(40),activation='relu'))
#     model3.add(Dense(int(10),activation='relu'))
#     model3.add(Dense(1,activation='sigmoid'))
#     print(model3.summary())
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath=Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='accuracy',mode='auto',patience=50,verbose=1)
#     history = model.fit(XTRAIN,
#                     YTRAIN,
#                     validation_data=(XVALIDATION,YVALIDATION),
#                     epochs=6000,
#                     batch_size=128*500,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
    

## Best Model

In [ ]:
# if True:
#     IN_DIM=len(mean)
#     model = Sequential()
#     model.add(Dense(int(IN_DIM/1),input_dim=IN_DIM,activation='relu'))
#     model.add(Dense(int(IN_DIM/1),activation='softplus'))
#     model.add(Dense(int(IN_DIM/3),activation='relu'))
#     model.add(Dense(int(IN_DIM/3),activation='relu'))
#     model.add(Dense(int(IN_DIM/6),activation='relu'))
#     model.add(Dense(1,activation='sigmoid'))
#     print(model.summary())
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath =MODEL_FILE,monitor ='val_loss',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='val_loss',mode='auto',patience=30,verbose=1)
#     history = model.fit(XTRAIN,
#                     YTRAIN,
#                     validation_data=(XVALIDATION,YVALIDATION),
#                     epochs=6000,
#                     batch_size=128*500,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#     #94%

In [ ]:
# ## Mega fast import
# import numba as nb
# import numpy as np
# import os
# os.environ[ 'NUMBA_CACHE_DIR' ] = '/UltimeTradingBot/Data/csv/tmp/'

# @nb.njit('float64[:,:](uint8[::1],)', cache=True)
# def decode_csv_buffer(rawData):
#     COMMA = np.uint8(ord(','))
#     CR = np.uint8(ord('\r'))
#     LF = np.uint8(ord('\n'))
#     ZERO = np.uint8(ord('0'))

#     # Find the size of the matrix (`n`)

#     n = 0
#     lineSize = 0

#     for i in range(rawData.size):
#         c = rawData[i]
#         if c == CR or c == LF:
#             break
#         n += rawData[i] == COMMA
#         lineSize += 1
    
#     n += 1

#     # Empty matrix
#     if lineSize == 0:
#         return np.empty((0, 0), dtype=np.float64)

#     # Initialization

#     res = np.empty(n * n, dtype=np.float64)

#     # Fill the matrix

#     curInt = 0
#     curPos = 0
#     lastCharIsDigit = True

#     for i in range(len(rawData)):
#         c = rawData[i]
#         if c == CR or c == LF or c == COMMA:
#             if lastCharIsDigit:
#                 # Write the last int in the flatten matrix
#                 res[curPos] = curInt
#                 curPos += 1
#                 curInt = 0
#             lastCharIsDigit = False
#         else:
#             curInt = curInt * 10 + (c - ZERO)
#             lastCharIsDigit = True

#     return res.reshape(n, n)
# def load_numba(filename):
#     # Load fully the file in a raw memory buffer
#     rawData = np.fromfile(filename, dtype=np.uint8)

#     # Decode the buffer using the Numba JIT
#     # This method only work for your specific needs and 
#     # can simply crash if the file content is invalid.
#     return decode_csv_buffer(rawData)

In [ ]:
# filename='/UltimeTradingBot/Data/csv/W20F4minP35.csv'

# dt=load_numba(filename)

In [ ]:
# if not TESTING_MOD:
#     IN_DIM=len(mean)
#     model = Sequential()
#     model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='softplus'))
#     model.add(Dense(int(IN_DIM/2),activation='relu'))
#     model.add(Dropout(0.3))
#     model.add(Dense(int(IN_DIM/5),activation='softplus'))
#     model.add(Dense(int(IN_DIM/4),activation='softmax'))
#     model.add(Dense(int(IN_DIM/1),activation='softplus'))
#     model.add(Dense(1,activation='sigmoid'))
#     print(model.summary())
#     #model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
#     # model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     # callbacks_a = ModelCheckpoint(filepath=Model_FileName+'.sftp.hdf5',monitor ='val_accuracy',save_best_only = True, save_weights = True)
#     # callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=10,verbose=1)
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath=Model_FileName+'.sftp.hdf5',monitor ='val_accuracy',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=10,verbose=1)
#     history = model.fit(XTRAIN,
#                     YTRAIN,
#                     validation_data=(XVALIDATION,YVALIDATION),
#                     epochs=6000,
#                     batch_size=128*500,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#     #94%

In [ ]:
print('##########################################################################')
print(f'Input shape: {XVALIDATION.shape}')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")


In [ ]:
#dt=df.to_numpy()
#if TESTING_MOD:
normalize(dt[:,0:-1])

In [ ]:
Normalization

In [ ]:
from keras.models import load_model

model = load_model(Model_FileName)

print(Model_FileName+' Loaded')

In [ ]:
eth.iloc[:,:-8].reset_index().reset_index().drop(columns="date")

In [ ]:
dteth=eth.iloc[:,:-8].reset_index().reset_index().drop(columns="date").to_numpy()

In [ ]:
peth=model.predict(dteth)

In [ ]:

#import tensorflow as tf
dt=np.nan_to_num(dt)

dt=tf.convert_to_tensor(dt, dtype=tf.float32)
#dt==dt.astype(np.float64)


In [ ]:
gc.collect()

In [ ]:


XVALIDATION[YVALIDATION==1]

## Retrain

In [ ]:
XX0=XVALIDATION[YVALIDATION==0]
YY0=YVALIDATION[YVALIDATION==0]

In [ ]:
INDEX_SEP=int(XX0.shape[0]/4)
XX0Train=XX0[INDEX_SEP:]
YY0Train=YY0[INDEX_SEP:]

XX0Val=XX0[:INDEX_SEP]
YY0Val=YY0[:INDEX_SEP]

In [ ]:
XX1=XVALIDATION[YVALIDATION==1]
YY1=YVALIDATION[YVALIDATION==1]

INDEX_SEP=int(XX1.shape[0]/4)
XX1Train=XX1[INDEX_SEP:]
YY1Train=YY1[INDEX_SEP:]

XX1Val=XX1[:INDEX_SEP]
YY1Val=YY1[:INDEX_SEP]

In [ ]:
accuracy0 = model.evaluate(XX0, YY0)
accuracy1 = model.evaluate(XX1, YY1)
accuracy = model.evaluate(XVALIDATION, YVALIDATION)

print(f"class 0: {format(accuracy0[1]*100,'0.2f')} %")
print(f"class 1: {format(accuracy1[1]*100,'0.2f')} %")
print(f"FULL class : {format(accuracy[1]*100,'0.2f')} %")

#accuracy = model.evaluate(dt[:,0:-1], dt[:,-1])


In [ ]:
accuracy = model.evaluate(XTRAIN, YTRAIN)


In [ ]:
print(f"class 0: {format(accuracy0[1]*100,'0.2f')} %")
print(f"class 1: {format(accuracy1[1]*100,'0.2f')} %")
print(f"FULL class : {format(accuracy[1]*100,'0.2f')} %")


In [ ]:
model.save(Model_FileName)

In [ ]:
Model_FileName

In [ ]:
XXXX=np.concatenate((XTRAIN,XX0Train))
YYYY=np.concatenate((YTRAIN,YY0Train))


In [ ]:
XXXX.shape

In [ ]:
# if True:
#     IN_DIM=len(mean)
#     model = Sequential()
#     model.add(Dense(int(IN_DIM/1),input_dim=IN_DIM,activation='relu'))
#     model.add(Dense(int(IN_DIM/1),activation='softplus'))
#     model.add(Dense(int(IN_DIM/3),activation='relu'))
#     model.add(Dense(int(IN_DIM/3),activation='relu'))
#     model.add(Dense(int(IN_DIM/6),activation='relu'))
#     model.add(Dense(1,activation='sigmoid'))
#     print(model.summary())
#     model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath =MODEL_FILE+".z.h5",monitor ='val_loss',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='val_loss',mode='auto',patience=5,verbose=1)
#     history = model.fit(XXXX,
#                     YYYY,
#                     validation_data=(XX0Val,YY0Val),
#                     epochs=6000,
#                     batch_size=128*500,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#     #94%

In [ ]:
# if True:
#     IN_DIM=len(mean)
#     model2 = Sequential()
#     model2.add(Dense(int(IN_DIM/1),input_dim=IN_DIM,activation='tanh'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(IN_DIM/1),activation='relu'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(IN_DIM/3),activation='relu'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(IN_DIM/3),activation='relu'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(IN_DIM/12),activation='relu'))
#     model2.add(Dense(1,activation='sigmoid'))
#     print(model2.summary())
#     model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath =MODEL_FILE+".z2.h5",monitor ='val_loss',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='val_loss',mode='auto',patience=5,verbose=1)
#     history = model2.fit(XXXX,
#                     YYYY,
#                     validation_data=(XX0Val,YY0Val),
#                     epochs=6000,
#                     batch_size=55555,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#     #94%

In [ ]:
# if True:
#     IN_DIM=len(mean)
#     model2 = Sequential()
#     model2.add(Dense(int(IN_DIM),input_dim=IN_DIM,activation='tanh'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(100),activation='relu'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(50),activation='relu'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(20),activation='relu'))
#     model2.add(Dropout(0.3))
#     model2.add(Dense(int(10),activation='relu'))
#     model2.add(Dense(1,activation='sigmoid'))
#     print(model2.summary())
#     model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#     callbacks_a = ModelCheckpoint(filepath =MODEL_FILE+".z4.h5",monitor ='val_loss',save_best_only = True, save_weights = True)
#     callbacks_b = EarlyStopping(monitor ='val_loss',mode='auto',patience=5,verbose=1)
#     history = model2.fit(XXXX,
#                     YYYY,
#                     validation_data=(XX0Val,YY0Val),
#                     epochs=6000,
#                     batch_size=55555,
#                     callbacks=[callbacks_a,callbacks_b])

#     print('##########################################################################')
#     print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#     #94%

In [ ]:
# retest
accuracy0 = model2.evaluate(XX0, YY0)
accuracy1 = model2.evaluate(XX1, YY1)
accuracy = model2.evaluate(XVALIDATION, YVALIDATION)

print(f"class 0: {accuracy0[1]*100}%")
print(f"class 1: {accuracy1[1]*100}%")
print(f"FULL Model : {accuracy[1]*100}%")

#accuracy = model.evaluate(dt[:,0:-1], dt[:,-1])


### Use the model

In [ ]:

# # make probability predictions with the model
# predictions = model.predict(dt[0:,0:-1])
# # round predictions 
# rounded = [round(x[0]) for x in predictions]

In [ ]:
sum(rounded)/len(rounded)

In [ ]:
YVALIDATION[1002]

In [12]:
def Buy_Dessision(input):
    predictions = model.predict(XVALIDATION)
    rounded = [round(x[0]) for x in predictions]

In [13]:
# Costumaize buy condition here
def buy_only(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    try:
        
        ## test param
        #buy_pourcent=1
        #window=15
        print (f"---buy_only--- Buy percent: {buy_pourcent}%")
        mino=buy_pourcent*0.01
        maxo=-sell_pourcent*0.01
        codep1='df["b"]=((('
        for i in range(1,window):
            codep1=codep1+'df["high"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
        codep2='df["high"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        
        codep1='df["sell"]=((( '
        for i in range(1,window):
            codep1=codep1+'df["low"].shift(periods='+str(-i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo ) | (('
        codep2='df["low"].shift(periods='+str(-window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
        code=codep1+codep2
        prerr(code)
        exec(code)

        df["buy"]=((df['b']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
        
        

    except Exception as e:
        print("Error buy only")
        print(e)
    p=df.pop("b")
    p=df.pop("sell")

    return df

In [14]:
def mini_expand3(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=3,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='1 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='5 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list15m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='15 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1h["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 H'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1d["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 d'):Pair_Full.iloc[len(Pair_Full)-1].name],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    # Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
    #         right_index=True, suffixes=('', ''))
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='inner',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    #buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    buy_only(Merged,buy_pourcent=buy_pourcent,sell_pourcent=sell_pourcent,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    # Merged=justlast_remover(Merged)
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=-100*(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=-100*(Merged["price"]-Merged[key])/Merged["price"]
    Merged=Merged.dropna()
    return Merged

## 2- Garbage Test

In [15]:
for pair in Binance_USDT_HALAL:
    price_volatility_5m=(100*(df_list5m[pair]["high"]-df_list5m[pair]["low"])/df_list5m[pair]["high"]).mean()

In [16]:
pair="SOL/USDT"
price_volatility_5m=(100*(df_list5m[pair]["high"]-df_list5m[pair]["low"])/df_list5m[pair]["high"]).mean()
print(price_volatility_5m)

0.6588983589901976


In [17]:
df=mini_expand3(pair=pair,i=0,j=len(df_list1m[pair]),window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=max(price_volatility_5m*0.6,0.8),sell_pourcent=SELL_PERCENT)

---buy_only--- Buy percent: 0.8%


In [18]:
df.pop("price")
df.pop("BTC_price")

date
2020-08-21 00:00:00    11851.216667
2020-08-21 00:01:00    11852.235000
2020-08-21 00:02:00    11866.776667
2020-08-21 00:03:00    11874.653333
2020-08-21 00:04:00    11871.078333
                           ...     
2022-05-18 13:56:00    29416.931667
2022-05-18 13:57:00    29379.176667
2022-05-18 13:58:00    29355.188333
2022-05-18 13:59:00    29370.298333
2022-05-18 14:00:00    29365.115000
Name: BTC_price, Length: 913897, dtype: float64

In [19]:
print(df.buy.mean()*100)

2.254192759140253


In [20]:
df

,high-1,low-1,close-1,volume-1,high-2,low-2,close-2,volume-2,high-3,low-3,...,BTC_volume-9_5min,BTC_high-10_5min,BTC_low-10_5min,BTC_close-10_5min,BTC_volume-10_5min,day,hour,minute,lunch_day,buy
date,,,,,,,,,,,,,,,,,,,,,
2020-08-21 00:00:00,-0.045238,-0.045238,-0.045238,0.00,-0.045238,-0.045238,-0.045238,365.46,-0.042118,-0.042118,...,64.780713,-0.049756,-0.115319,-0.074563,126.811156,5,0,0,-223,0
2020-08-21 00:01:00,0.129063,0.038511,0.038511,474.40,0.038511,0.038511,0.038511,0.00,0.038511,0.038511,...,64.780713,-0.058343,-0.123901,-0.083149,126.811156,5,0,1,-223,0
2020-08-21 00:02:00,-0.038481,-0.153925,-0.153925,1272.74,0.052002,-0.038481,-0.038481,474.40,-0.038481,-0.038481,...,64.780713,-0.180813,-0.246290,-0.205588,126.811156,5,0,2,-223,0
2020-08-21 00:03:00,0.020789,-0.213087,0.020789,513.37,-0.094590,-0.209968,-0.209968,1272.74,-0.004158,-0.094590,...,64.780713,-0.247025,-0.312458,-0.271783,126.811156,5,0,3,-223,0
2020-08-21 00:04:00,0.149743,-0.131025,-0.131025,170.09,0.062393,-0.171580,0.062393,513.37,-0.053034,-0.168460,...,64.780713,-0.216984,-0.282437,-0.241750,126.811156,5,0,4,-223,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-18 13:56:00,0.312651,0.041902,0.157937,4598.55,0.448026,0.080580,0.080580,3694.60,0.390008,-0.170830,...,306.828060,0.548250,0.131007,0.341430,853.220830,3,13,56,-223,0
2022-05-18 13:57:00,0.339114,-0.125957,-0.029067,8846.37,0.513516,0.242225,0.358492,4598.55,0.649162,0.280981,...,306.828060,0.677464,0.259685,0.470379,853.220830,3,13,57,-223,0
2022-05-18 13:58:00,0.255424,-0.132562,-0.054965,3061.43,0.449416,-0.016166,0.080830,8846.37,0.624010,0.352420,...,306.828060,0.759735,0.341615,0.552480,853.220830,3,13,58,-223,0


In [21]:
df=df.reset_index()
try:df.pop("num_index")
except: pass
try:df.pop("index")
except: pass
try:df.pop("date")
except: pass
df=data_shufler(df)            
df=data_chooser(df,weight=50,row_numbers=df.buy.sum()*2)
gc.collect()
df=data_cleanup(df)

In [22]:
(df.shape[0]/2)==df.buy.sum()

True

In [23]:
dt=df.to_numpy(dtype=np.float64)

In [24]:
dt[0,0]

0.3307487935802198

In [25]:
index_20percent= int(0.1*len(dt[:,0]))
print(index_20percent)
XVALIDATION= dt[:index_20percent, :-1]
YVALIDATION= dt[:index_20percent,-1]
XTRAIN= dt[index_20percent:, 0:-1]
YTRAIN= dt[index_20percent:,-1]

4120


In [26]:
XVALIDATION[0,0]

0.3307487935802198

In [27]:

if True:
    if FIRST_NORM_FLAG:
    #if True:
        print("normalizing ...")
        mean = XTRAIN.mean(axis=0)
        std = XTRAIN.std(axis=0)



        XTRAIN -= mean 
        XTRAIN /= std

        XVALIDATION -=mean
        XVALIDATION /= std
        FIRST_NORM_FLAG=False
        ######################### SAVIN NORM ################
        try:
            Normalization={"mean":mean.tolist(),"std":std.tolist()}
            with open(Normalization_File.replace("/UltimeTradingBot/Data/", "/UltimeTradingBot/Data/"+pair.replace("/", "-")+"-"), 'w+') as fp:
                        json.dump(Normalization, fp,  indent=4)
                        print(Normalization_File)
        except Exception as e:
            print("error Normalization in juppiter")
            print(e)
    else:print("already normalized")

normalizing ...
/UltimeTradingBot/Data/tp120_w10_max5min_Norm_v1.json


In [28]:
Y=dt[:,-1]

In [29]:
IN_DIM=dt.shape[1]-1
model = Sequential()
model.add(Dense(int(IN_DIM),input_dim=IN_DIM,activation='relu')) #( 100=>66.23)
# resultus withe 250 d0.3 50 d 20 d 8 = 65.6% vs  65.49 no droppout vs 65.78 d0.5 vs 65.68 d0.4 # 65.48 # one dropout of   0.5 = 66.11   #tanh 66.33 accuracy #softmax 66.12 #softplus 66.5 # sigmoid 66.21 / 
model.add(Dropout(0.5))
model.add(Dense(int(20),activation='relu')) # best softplus 66.26(69) vs relu 66.65(70)  / -20 => 66.25 /250 ->65.78
#model2.add(Dropout(0.1)) #5=> 66.00  66.21 vs no dopout 66.28
#model2.add(Dense(int(20),activation='relu')) # disabled 6.24
model.add(Dense(int(50),activation='relu')) # -4 -> -> 66.24
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath =MODEL_FILE+f"test.multi_train.h5",monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=15,verbose=1)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:,-1],
                validation_data=(dt[:index_20percent, :-1],Y[:index_20percent]),
                epochs=6000,
                batch_size=1000,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
#94%

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 404)               163620    
                                                                 
 dropout (Dropout)           (None, 404)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                8100      
                                                                 
 dense_2 (Dense)             (None, 50)                1050      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 172,821
Trainable params: 172,821
Non-trainable params: 0
_________________________________________________________________


2022-11-11 22:46:14.452346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-11 22:46:14.452417: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-11 22:46:14.452466: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: abj-K93SV
2022-11-11 22:46:14.452485: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: abj-K93SV
2022-11-11 22:46:14.452803: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2022-11-11 22:46:14.452911: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 390.154.0


None
Epoch 1/6000
38/38 [==============================] - 2s 24ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 2/6000
38/38 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 3/6000
38/38 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 4/6000
38/38 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 5/6000
38/38 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 6/6000
38/38 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 7/6000
38/38 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.4922
Epoch 8/6000
38/38 [================